# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [12]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json 
import os


# Import API key
from api_keys import geoapify_key

In [13]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,São Filipe,14.8961,-24.4956,70.02,60,41,16.69,CV,1675707329
1,1,Paamiut,61.9940,-49.6678,12.69,68,100,10.00,GL,1675707708
2,2,Punta Arenas,-53.1500,-70.9167,64.51,45,75,33.38,CL,1675707298
3,3,Hofn,64.2539,-15.2082,33.82,93,100,5.75,IS,1675707618
4,4,Hambantota,6.1241,81.1185,76.23,88,88,10.25,LK,1675707344


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [14]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Distance",
    scale = 0.5,
    color = "City"
)

map_plot




:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE

vacation_df = city_data_df[(city_data_df["Max Temp"] >70) & (city_data_df["Max Temp"] <80) \
                            & (city_data_df["Wind Speed"] <10) & (city_data_df["Cloudiness"] ==0)]\
                              .dropna()


# Drop any rows with null values
# YOUR CODE HERE

# Display sample data
# YOUR CODE HERE

vacation_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
151,151,Cabo San Lucas,22.8909,-109.9124,79.52,47,0,4.61,MX,1675707735
395,395,Tamiahua,21.2667,-97.4500,75.74,66,0,8.08,MX,1675707796
434,434,Warin Chamrap,15.1932,104.8628,75.02,88,0,1.14,TH,1675707804
460,460,St. Marys,30.7305,-81.5465,72.10,50,0,4.00,US,1675707683
485,485,Lakeside,30.1300,-81.7682,73.54,42,0,8.05,US,1675707818
494,494,Tallahassee,30.4383,-84.2807,71.10,43,0,6.91,US,1675707530
538,538,El Rodeo (El Naranjo),21.4667,-98.6000,78.13,46,0,4.41,MX,1675707832
544,544,Ixtapa,20.7000,-105.2000,78.03,47,0,4.61,MX,1675707567


### Step 3: Create a new DataFrame called `hotel_df`.

In [16]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE

hotel_df = vacation_df.loc[:,["City", "Country", "Lat", "Lng"]]




# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE

hotel_df["Hotel Name"] = ""

# Display sample data
# YOUR CODE HERE

hotel_df



,City,Country,Lat,Lng,Hotel Name
151,Cabo San Lucas,MX,22.8909,-109.9124,
395,Tamiahua,MX,21.2667,-97.4500,
434,Warin Chamrap,TH,15.1932,104.8628,
460,St. Marys,US,30.7305,-81.5465,
485,Lakeside,US,30.1300,-81.7682,
494,Tallahassee,US,30.4383,-84.2807,
538,El Rodeo (El Naranjo),MX,21.4667,-98.6000,
544,Ixtapa,MX,20.7000,-105.2000,


In [25]:
radius = 10000
params = {}
params['apiKey'] = geoapify_key
params['categories'] = 'accommodation.hotel'
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    latitude = hotel_df.loc[index, 'Lat']
    longitude = hotel_df.loc[index, 'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

SyntaxError: invalid non-printable character U+00A0 (2699328770.py, line 10)

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [26]:
# Set parameters to search for a hotel
radius = 10000
params = {}
params['apiKey'] = geoapify_key
params['categories'] = 'accommodation.hotel'

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    
    
    lat = hotel_df.loc[index, 'Lat']
    lng = hotel_df.loc[index, 'Lng']
   
    
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Cabo San Lucas - nearest hotel: Comfort Rooms
Tamiahua - nearest hotel: No hotel found
Warin Chamrap - nearest hotel: ดารุณี
St. Marys - nearest hotel: No hotel found
Lakeside - nearest hotel: Hampton Inn
Tallahassee - nearest hotel: The Governors Inn
El Rodeo (El Naranjo) - nearest hotel: No hotel found
Ixtapa - nearest hotel: Ma. Cristina


,City,Country,Lat,Lng,Hotel Name
151,Cabo San Lucas,MX,22.8909,-109.9124,Comfort Rooms
395,Tamiahua,MX,21.2667,-97.4500,No hotel found
434,Warin Chamrap,TH,15.1932,104.8628,ดารุณี
460,St. Marys,US,30.7305,-81.5465,No hotel found
485,Lakeside,US,30.1300,-81.7682,Hampton Inn
494,Tallahassee,US,30.4383,-84.2807,The Governors Inn
538,El Rodeo (El Naranjo),MX,21.4667,-98.6000,No hotel found
544,Ixtapa,MX,20.7000,-105.2000,Ma. Cristina


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [44]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    hover_cols = ["Country", "Hotel Name"],
    scale = 0.5
    
    
)

# Display the map
# YOUR CODE HERE

map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Country,Hotel Name)